# Пипы

In [ ]:
pip install --upgrade nbformat


# Код

In [ ]:
import pandas as pd
import os
import glob

# Путь к папке с CSV файлами
folder_path = '/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/Andrey Simonov/useful users'

# Используем glob для получения всех файлов с расширением .csv в папке
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
csv_files = csv_files#[0:2]
# Загрузка таблицы пользователей и групп (используем pandas)
user_groups = pd.read_csv('user|group just FEM subs.csv', dtype={'id': 'int64'}, usecols=['id'])
merged = user_groups.copy()  # just the id for future processing ease
trial = 1
# Пройдем по всем файлам CSV в папке
for file in csv_files:
    print("Processing file ", file)
    # Загрузка характеристик пользователей
    user_features = pd.read_csv(file, dtype={'id': 'int64'},low_memory=False)
    
    # Объединение таблиц по столбцу 'id' с добавлением суффиксов для предотвращения дублирования
    merged = merged.merge(user_features, how='left', on='id', suffixes=('', '_new'))
    
    # Проходим по всем столбцам с суффиксами и объединяем их
    for column in user_features.columns:
        if column != 'id':  # Пропускаем столбец 'id'
            if column + '_new' in merged.columns:
                # Заполняем NaN значениями из _new, если столбец существует
                merged[column] = merged[column].fillna(merged[column + '_new'])
                merged = merged.drop(columns=[column + '_new'])  # Убираем временный столбец с суффиксом

    # Если нужно, можно также переименовать оставшиеся столбцы, чтобы убрать суффикс
    merged = merged.rename(columns=lambda x: x.replace('_new', ''))

# Сохранение итогового результата в CSV
merged.to_csv(f'Merged_fem_groups_with_user_chars_{trial}.csv', index=False)




In [ ]:
user_groups = pd.read_csv('user|group just FEM subs.csv', dtype={'id': 'int64'})
merged = merged.merge(user_groups, how='left', on='id')

Теперь у меня есть таблица с юзерами группами и характеристиками юзеров (напомню, юзеры по строкам). Мне нужно сделать сводную таблицу, которая показывает, сколько у каждой группы юзеров из каждого города (city.id), отдельно такая же таблица для каждого пола (sex), а также похожие саммари таблички по полям 'personal.alcohol', 'personal.langs', 'personal.life_main', 'personal.people_main', 'personal.political', 'personal.religion', 'personal.smoking'

In [ ]:
import pandas as pd
import vk_api
# Read the access token from the file
with open('/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/TOKEN VK') as f:
    access_token = f.read().strip()

# Initialize VK session
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()

In [ ]:
# Extract unique city IDs from the dataframe
unique_city_ids = merged['city.id'].dropna().unique()

In [ ]:
import pandas as pd
import vk_api
from tqdm import tqdm  # Для отображения прогресса
import time



# Fetch city names from VK API
city_mapping = {}
for city_id in tqdm(unique_city_ids, desc="Fetching city names"):
    try:
        # Call VK API to get city details
        response = vk.database.getCitiesById(city_ids=city_id)
        if response:
            city_mapping[city_id] = response[0]['title']  # Extract city name
        else:
            city_mapping[city_id] = None  # Handle case where city is not found
    except Exception as e:
        print(f"Error fetching city ID {city_id}: {e}")
        city_mapping[city_id] = None
    #time.sleep(0.35)  # Pause to respect VK API rate limits




In [ ]:
# Replace city IDs with names in the dataframe
merged['city.name'] = merged['city.id'].map(city_mapping)

In [ ]:
# Save the updated dataframe to a file (optional)
merged.to_csv('Merged_fem_groups_with_user_chars_city_names.csv', index=False)

# Preview the updated dataframe
print(merged.head())


### Если тут работу остановили, то можно заново загрузить merged уже из csv

In [ ]:
import pandas as pd
merged = pd.read_csv('Merged_fem_groups_with_user_chars_city_names.csv',  encoding='utf-8')

### Отбросим группы, не подходящие (не фем/локальные)

In [ ]:
merged = merged.drop(columns=['80728963', '153276634', '189672959'])

Остались колонки 'id', 'bdate', 'country', 'facebook', 'facebook_name', 'first_name',
       'home_town', 'instagram', 'last_name', 'livejournal', 'screen_name',
       'sex', 'twitter', 'city.id', 'city.title', 'personal.alcohol',
       'personal.inspired_by', 'personal.langs', 'personal.life_main',
       'personal.people_main', 'personal.political', 'personal.religion',
       'personal.smoking', '81914161', '190884646', '96917976', '204732875',
       '198043329', '159437367', '74599974', '195772623', '73197614',
       '73837757', '87819738', '114487519', '65878810', '128501486',
       '204172284', '209222517', '189672959', '163233491', '225770630',
       'city.name'